# Preparar Datos

In [29]:
import pandas as pd
from geopy.geocoders import Nominatim

In [18]:
df = pd.read_csv("./ARS_Base-inicial_MIAD.csv", decimal='.', thousands=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 34 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Organization Name                             2211 non-null   object 
 1   Headquarters Location                         2185 non-null   object 
 2   Operating Status                              2211 non-null   object 
 3   Company Type                                  1409 non-null   object 
 4   Number of Investments                         1494 non-null   float64
 5   Number of Lead Investments                    1348 non-null   float64
 6   Number of Diversity Investments               1059 non-null   float64
 7   Number of Exits                               1207 non-null   float64
 8   Number of Lead Investors                      1088 non-null   float64
 9   Number of Investors                           1195 non-null   f

## 1. Limpiar Data

### 1.1 Tomar columnas relevantes

In [19]:
df = df[['Organization Name', 'Headquarters Location', 'Operating Status',
       'Company Type', 'Number of Investments', 'Number of Lead Investments',       
       'Number of Diversity Investments', 'Number of Exits',
       'Number of Lead Investors', 'Number of Investors',
       # Necesitamos saber como usar estos campos
       # 'Industry Groups', 'Industries',
       'Number of Funding Rounds', 'Funding Status',
       'Last Funding Date', 
       # Valores redundantes
       # 'Last Funding Amount', 'Last Funding Amount Currency',
       'Last Funding Amount Currency (in USD)',
       'Last Funding Type', 'Last Equity Funding Type',
       # Valores redundantes
       # 'Last Equity Funding Amount', 'Last Equity Funding Amount Currency',
       'Last Equity Funding Amount Currency (in USD)',
       # Valores redundantes
       # 'Total Funding Amount', 'Total Funding Amount Currency',
       'Total Funding Amount Currency (in USD)', 'Number of Events',
       # Se necesita entender estos valores y como convertirlos
       # 'SEMrush - Monthly Visits', 'SEMrush - Average Visits (6 months)', 'SEMrush - Visit Duration', 
       # Valores redundantes
       # 'Aberdeen - IT Spend', 'Aberdeen - IT Spend Currency',
       'Aberdeen - IT Spend Currency (in USD)',
       # Utilizado para hacer arcos
       # 'Principales inversionistas'
    ]]

len(df.columns)

20

In [20]:
df.rename(columns={
    'Last Funding Amount Currency (in USD)': 'Last Funding Amount',
    'Last Equity Funding Amount Currency (in USD)': 'Last Equity Funding Amount',
    'Total Funding Amount Currency (in USD)': 'Total Funding Amount',
    'Aberdeen - IT Spend Currency (in USD)': 'Aberdeen - IT Spend'
}, inplace=True)

df.columns

Index(['Organization Name', 'Headquarters Location', 'Operating Status',
       'Company Type', 'Number of Investments', 'Number of Lead Investments',
       'Number of Diversity Investments', 'Number of Exits',
       'Number of Lead Investors', 'Number of Investors',
       'Number of Funding Rounds', 'Funding Status', 'Last Funding Date',
       'Last Funding Amount', 'Last Funding Type', 'Last Equity Funding Type',
       'Last Equity Funding Amount', 'Total Funding Amount',
       'Number of Events', 'Aberdeen - IT Spend'],
      dtype='object')

### 1.2 Reemplazar valor null 

In [21]:
# Campos númericos

numeric_columns = df.select_dtypes(include='number')
numeric_columns_filled = numeric_columns.fillna(0)
df[numeric_columns_filled.columns] = numeric_columns_filled

In [22]:
df['Headquarters Location'] = df['Headquarters Location'].fillna('None, None, None')
df['Company Type'] = df['Company Type'].fillna('Non Profit')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Organization Name                2211 non-null   object 
 1   Headquarters Location            2211 non-null   object 
 2   Operating Status                 2211 non-null   object 
 3   Company Type                     2211 non-null   object 
 4   Number of Investments            2211 non-null   float64
 5   Number of Lead Investments       2211 non-null   float64
 6   Number of Diversity Investments  2211 non-null   float64
 7   Number of Exits                  2211 non-null   float64
 8   Number of Lead Investors         2211 non-null   float64
 9   Number of Investors              2211 non-null   float64
 10  Number of Funding Rounds         2211 non-null   float64
 11  Funding Status                   1131 non-null   object 
 12  Last Funding Date   

### 1.3 Dividir location

In [24]:
df1 = df.copy()
split_columns = df['Headquarters Location'].str.split(',', expand=True)
df1[['City', 'State', 'Country']] = split_columns[[0, 1, 2]]
_ = df1.drop(columns=['Headquarters Location'], inplace=True)

In [25]:
### 1.4 Convertir el nombre de la organización en label e Id

In [26]:
df = df1
df.rename(columns={'Organization Name': 'Id'}, inplace=True)
df['Label'] = df['Id']

# Reordena valores
columns = df.columns.tolist()
first_cols = ['Id', 'Label', 'City', 'State', 'Country']
new_order = first_cols + [col for col in columns if col not in first_cols]
df = df[new_order]
df.columns

Index(['Id', 'Label', 'City', 'State', 'Country', 'Operating Status',
       'Company Type', 'Number of Investments', 'Number of Lead Investments',
       'Number of Diversity Investments', 'Number of Exits',
       'Number of Lead Investors', 'Number of Investors',
       'Number of Funding Rounds', 'Funding Status', 'Last Funding Date',
       'Last Funding Amount', 'Last Funding Type', 'Last Equity Funding Type',
       'Last Equity Funding Amount', 'Total Funding Amount',
       'Number of Events', 'Aberdeen - IT Spend'],
      dtype='object')

### 1.5 Obtener Lat y Long para la ubicación

In [62]:
loc = Nominatim(user_agent="Geopy Library")
dic_country = {}
dic_states = {}
dic_cities = {}
# entering the location name

def get_coor(dir, dic):
    coor = None
    if dir in dic:
        coor = dic[dir]
    else:
        l = loc.geocode(dir)
        if (l):
            coor = l.latitude, l.longitude
            dic[dir] = coor
        else:
            print('Dir no encontrada', dir, l)

    return coor

#### Coordenadas de país

In [55]:
df1 = df
df1[['Lat Country', 'Long Country']] = df1['Country'].apply(lambda x: pd.Series(get_coor(x, dic_country)))
# df1[0:2]['Country'].apply(lambda x: pd.Series(get_coor(x, dic_country)))


#### Coordenadas de estado

In [63]:
df = df1

states = df['Country'].astype(str) + ", " + df['State'].astype(str)
df[['Lat State', 'Long State']] = states.apply(lambda x: pd.Series(get_coor(x, dic_states)))

Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  Slovenia,  Ljubljana Urban Commune None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Kyonggi-do None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Kyonggi-do None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  Qatar,  Ad Dawhah None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None
Dir no encontrada  South Korea,  Seoul-t'ukpyolsi None


Se ajustan nombres de estado

In [76]:
df1 = df

to_replace = {
    " Seoul-t'ukpyolsi": "Seoul",
    " Ljubljana Urban Commune": "Ljubljana",
    " Kyonggi-do": "Gyeonggi-do",
    " Ad Dawhah": "Doha"

}
df1['State'].replace(to_replace, inplace=True)

states =  df1['State'].astype(str) + ", " + df1['Country'].astype(str)
df1[['Lat State', 'Long State']] = states.apply(lambda x: pd.Series(get_coor(x, dic_states)))



#### Coordenadas de ciudad

### 1.6 Guardar xls

In [80]:
df1.to_excel("nodos_limpios.xlsx", index=False)